In [1]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import xgboost as xg
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from scipy.stats import loguniform, randint
from sklearn.model_selection import RandomizedSearchCV, KFold

In [2]:
aqi_2008= pd.read_csv("2008 Air Quality Data.csv")
aqi_2009= pd.read_csv("2009 Air Quality Data.csv")
aqi_2010= pd.read_csv("2010 Air Quality Data.csv")
aqi_2011= pd.read_csv("2011 Air Quality Data.csv")
aqi_2012= pd.read_csv("2012 Air Quality Data.csv")
aqi_2013= pd.read_csv("2013 Air Quality Data.csv")
aqi_2014= pd.read_csv("2014 Air Quality Data.csv")
aqi_2015= pd.read_csv("2015 Air Quality Data.csv")
aqi_2016= pd.read_csv("2016 Air Quality Data.csv")
aqi_2017= pd.read_csv("2017 Air Quality Data.csv")
aqi_2018= pd.read_csv("2018 Air Quality Data.csv")
aqi_2019= pd.read_csv("2019 Air Quality Data.csv")
aqi_2020= pd.read_csv("2020 Air Quality Data.csv")
aqi_2021= pd.read_csv("2021 Air Quality Data.csv")
aqi_2022= pd.read_csv("2022 Air Quality Data.csv")
aqi_2023= pd.read_csv("2023 Air Quality Data.csv")
aqi_2024= pd.read_csv("2024 Air Quality Data.csv")

gas_price = pd.read_csv("Gas Prices.csv")
ZEVLDV_data = pd.read_csv("ZEV+LDV Sales.csv")
ng_consumption = pd.read_csv("ng_consumption.csv")
elec_consumption = pd.read_csv("elec_consumption.csv")

In [3]:
total_aqi = pd.concat([aqi_2008, aqi_2009, aqi_2010, aqi_2011, aqi_2012, aqi_2013, aqi_2014,
                    aqi_2015, aqi_2016, aqi_2017, aqi_2018, aqi_2019, aqi_2020, aqi_2021, aqi_2022,
                    aqi_2023, aqi_2024], axis = 0, join='outer')
total_aqi = total_aqi[["Date", "County", "Daily Mean PM2.5 Concentration", "Daily AQI Value"]]
total_aqi['Date'] = pd.to_datetime(total_aqi['Date'])
total_aqi

,Date,County,Daily Mean PM2.5 Concentration,Daily AQI Value
0,2008-01-01,Alameda,25.7,82
1,2008-01-02,Alameda,30.1,90
2,2008-01-03,Alameda,13.5,59
3,2008-01-04,Alameda,2.6,14
4,2008-01-07,Alameda,5.9,33
...,...,...,...,...
58823,2024-12-27,Yolo,11.4,55
58824,2024-12-28,Yolo,8.8,49
58825,2024-12-29,Yolo,6.7,37
58826,2024-12-30,Yolo,7.4,41


In [4]:
counties = total_aqi['County'].unique()
counties.astype(list)

array(['Alameda', 'Butte', 'Calaveras', 'Colusa', 'Contra Costa',
       'Del Norte', 'El Dorado', 'Fresno', 'Glenn', 'Humboldt',
       'Imperial', 'Inyo', 'Kern', 'Kings', 'Lake', 'Los Angeles',
       'Marin', 'Mariposa', 'Mendocino', 'Merced', 'Mono', 'Monterey',
       'Napa', 'Nevada', 'Orange', 'Placer', 'Plumas', 'Riverside',
       'Sacramento', 'San Benito', 'San Bernardino', 'San Diego',
       'San Francisco', 'San Joaquin', 'San Luis Obispo', 'San Mateo',
       'Santa Barbara', 'Santa Clara', 'Santa Cruz', 'Shasta', 'Siskiyou',
       'Solano', 'Sonoma', 'Stanislaus', 'Sutter', 'Trinity', 'Tulare',
       'Ventura', 'Yolo', 'Madera', 'Tehama', 'Alpine'], dtype=object)

In [5]:


total_aqi['Date'] = pd.to_datetime(total_aqi['Date'], format='%b-%Y')
total_aqi['Fiscal_Year'] = total_aqi['Date'].apply(lambda x: x.year + 1 if x.month > 12 else x.year)
total_aqi['Fiscal_Quarter'] = total_aqi['Date'].apply(
    lambda x: 'Q1' if x.month in [1, 2, 3]
    else 'Q2' if x.month in [4, 5, 6]
    else 'Q3' if x.month in [7, 8, 9]
    else 'Q4'
)
total_aqi['County_Quarter'] = total_aqi['County'].astype(str) + "_" + total_aqi['Fiscal_Year'].astype(str) + total_aqi['Fiscal_Quarter']
total_aqi = total_aqi[["County_Quarter", "Daily AQI Value"]]
total_aqi_avg = (
    total_aqi
    .groupby('County_Quarter', as_index=False)['Daily AQI Value']
    .mean()
)

total_aqi_avg["County_Quarter"] = total_aqi_avg["County_Quarter"].str.upper()
total_aqi_avg

,County_Quarter,Daily AQI Value
0,ALAMEDA_2008Q1,40.465021
1,ALAMEDA_2008Q2,47.481481
2,ALAMEDA_2008Q3,49.563063
3,ALAMEDA_2008Q4,52.891051
4,ALAMEDA_2009Q1,40.027211
...,...,...
3447,YOLO_2023Q4,39.654639
3448,YOLO_2024Q1,22.896679
3449,YOLO_2024Q2,27.703704
3450,YOLO_2024Q3,38.584906


In [6]:
ZEVLDV_data

,Data Year,Quarter,COUNTY,ZEV Sales,Total LDV Sales,ZEV Sales Share
0,2025,1,Alameda,5525,14812,37%
1,2025,1,Alpine,0,3,0%
2,2025,1,Amador,31,234,13%
3,2025,1,Butte,118,1109,11%
4,2025,1,Calaveras,38,311,12%
...,...,...,...,...,...,...
4066,2008,4,Tulare,0,874,0%
4067,2008,4,Tuolumne,0,138,0%
4068,2008,4,Ventura,1,3649,0%
4069,2008,4,Yolo,0,961,0%


In [7]:
ZEVLDV_new = ZEVLDV_data[ZEVLDV_data["Data Year"] < 2025] 
ZEVLDV_new = ZEVLDV_new[ZEVLDV_new["COUNTY"] != "Out Of State"] 
ZEVLDV_new

,Data Year,Quarter,COUNTY,ZEV Sales,Total LDV Sales,ZEV Sales Share
59,2024,1,Alameda,5693,14884,38%
60,2024,1,Alpine,1,10,10%
61,2024,1,Amador,26,207,13%
62,2024,1,Butte,130,1106,12%
63,2024,1,Calaveras,40,297,13%
...,...,...,...,...,...,...
4066,2008,4,Tulare,0,874,0%
4067,2008,4,Tuolumne,0,138,0%
4068,2008,4,Ventura,1,3649,0%
4069,2008,4,Yolo,0,961,0%


In [8]:
ZEVLDV_new["County_Quarter"] = ZEVLDV_new["COUNTY"].astype(str) + "_" + ZEVLDV_new["Data Year"].astype(str) + "Q" + ZEVLDV_new["Quarter"].astype(str)

In [9]:
ZEVLDV_new = ZEVLDV_new[["County_Quarter", "ZEV Sales", "Total LDV Sales", "ZEV Sales Share"]]
ZEVLDV_new = ZEVLDV_new.rename(columns={"Total LDV Sales": "LDV Sales"})

In [10]:
ZEVLDV_new['County_Quarter'] = ZEVLDV_new['County_Quarter'].str.upper()
ZEVLDV_new

,County_Quarter,ZEV Sales,LDV Sales,ZEV Sales Share
59,ALAMEDA_2024Q1,5693,14884,38%
60,ALPINE_2024Q1,1,10,10%
61,AMADOR_2024Q1,26,207,13%
62,BUTTE_2024Q1,130,1106,12%
63,CALAVERAS_2024Q1,40,297,13%
...,...,...,...,...
4066,TULARE_2008Q4,0,874,0%
4067,TUOLUMNE_2008Q4,0,138,0%
4068,VENTURA_2008Q4,1,3649,0%
4069,YOLO_2008Q4,0,961,0%


In [11]:
elec_consumption 

,Year,CountyNum,CountyName,Sector,RNR,GWh
0,1990,1,ALAMEDA,Agriculture and Water Pumping,NR,903.919433
1,1990,1,ALAMEDA,Commercial,NR,3477.775257
2,1990,1,ALAMEDA,Industrial,NR,1895.868822
3,1990,1,ALAMEDA,Mining,NR,107.551622
4,1990,1,ALAMEDA,Residential,R,2498.265626
...,...,...,...,...,...,...
13755,2023,58,YUBA,Industrial,NR,14.240638
13756,2023,58,YUBA,Mining,NR,16.184693
13757,2023,58,YUBA,Residential,R,248.034236
13758,2023,58,YUBA,Streetlighting,NR,1.357392


In [12]:

elec_consumption = elec_consumption[elec_consumption["Year"] >= 2008] 
elec_consumption = elec_consumption[["Year","CountyName","GWh"]]

In [13]:
elec_consumption

,Year,CountyName,GWh
7285,2008,ALAMEDA,521.885222
7286,2008,ALAMEDA,5118.615553
7287,2008,ALAMEDA,1698.134132
7288,2008,ALAMEDA,61.144955
7289,2008,ALAMEDA,2993.082625
...,...,...,...
13755,2023,YUBA,14.240638
13756,2023,YUBA,16.184693
13757,2023,YUBA,248.034236
13758,2023,YUBA,1.357392


In [14]:
elec_new = elec_consumption.groupby(["Year", "CountyName"])["GWh"].sum().reset_index()
elec_new

,Year,CountyName,GWh
0,2008,ALAMEDA,11131.588895
1,2008,ALPINE,18.268355
2,2008,AMADOR,331.599609
3,2008,BUTTE,1452.147286
4,2008,CALAVERAS,319.693233
...,...,...,...
923,2023,TULARE,5786.611594
924,2023,TUOLUMNE,431.240845
925,2023,VENTURA,5095.898817
926,2023,YOLO,1692.563528


In [15]:
elec_expanded = elec_new.loc[elec_new.index.repeat(4)].reset_index(drop=True)
elec_expanded['Quarter'] = ['Q1', 'Q2', 'Q3', 'Q4'] * len(elec_new)
elec_expanded['County_Quarter'] = elec_expanded['CountyName'].astype(str) + "_" + elec_expanded['Year'].astype(str) + elec_expanded['Quarter']
elec_expanded = elec_expanded[["County_Quarter","GWh"]]
elec_expanded

,County_Quarter,GWh
0,ALAMEDA_2008Q1,11131.588895
1,ALAMEDA_2008Q2,11131.588895
2,ALAMEDA_2008Q3,11131.588895
3,ALAMEDA_2008Q4,11131.588895
4,ALPINE_2008Q1,18.268355
...,...,...
3707,YOLO_2023Q4,1692.563528
3708,YUBA_2023Q1,527.954870
3709,YUBA_2023Q2,527.954870
3710,YUBA_2023Q3,527.954870


In [16]:
ng_consumption.head()

,Year,CountyNum,CountyName,Sector,RNR,MMTherms
0,1990,ALAMEDA,1,Agriculture and Water Pumping,Non-Residentialidential,3.567820e+06
1,1990,ALAMEDA,1,Commercial,Non-Residentialidential,8.845901e+07
2,1990,ALAMEDA,1,Industrial,Non-Residentialidential,1.371807e+08
3,1990,ALAMEDA,1,Mining,Non-Residentialidential,1.187480e+06
4,1990,ALAMEDA,1,Residentialidential,Residential,2.712876e+08


In [17]:
ng_consumption = ng_consumption[ng_consumption["Year"] >= 2008] 
ng_consumption = ng_consumption[["Year","CountyNum"," MMTherms "]]

In [18]:
ng_new = ng_consumption.groupby(["Year", "CountyNum"])[" MMTherms "].sum().reset_index()
ng_new

,Year,CountyNum,MMTherms
0,2008,ALAMEDA,4.361089e+08
1,2008,AMADOR,8.402478e+06
2,2008,BUTTE,4.657998e+07
3,2008,CALAVERAS,1.130993e+06
4,2008,COLUSA,2.784985e+07
...,...,...,...
748,2023,TRINITY,4.958600e+04
749,2023,TULARE,1.661312e+08
750,2023,VENTURA,1.792170e+08
751,2023,YOLO,6.732702e+07


In [19]:
ng_expanded = ng_new.loc[ng_new.index.repeat(4)].reset_index(drop=True)
ng_expanded['Quarter'] = ['Q1', 'Q2', 'Q3', 'Q4'] * len(ng_new)
ng_expanded['County_Quarter'] = (
    ng_expanded['CountyNum'].astype(str) + "_" +
    ng_expanded['Year'].astype(str) + ng_expanded['Quarter']
)
ng_expanded = ng_expanded[["County_Quarter", " MMTherms "]]
ng_expanded

,County_Quarter,MMTherms
0,ALAMEDA_2008Q1,4.361089e+08
1,ALAMEDA_2008Q2,4.361089e+08
2,ALAMEDA_2008Q3,4.361089e+08
3,ALAMEDA_2008Q4,4.361089e+08
4,AMADOR_2008Q1,8.402478e+06
...,...,...
3007,YOLO_2023Q4,6.732702e+07
3008,YUBA_2023Q1,1.280382e+07
3009,YUBA_2023Q2,1.280382e+07
3010,YUBA_2023Q3,1.280382e+07


In [20]:
gas_price['Year'] = gas_price['Date'].str[-4:].astype(int)
gas_price_new = gas_price[gas_price['Year'] >= 2008]
gas_price_new = gas_price_new[gas_price_new['Year'] <= 2024]

In [21]:
gas_price_new

,Date,California All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon),Unnamed: 2,Unnamed: 3,Unnamed: 4,Year
92,Jan-2008,3.296,NaN,NaN,NaN,2008
93,Feb-2008,3.231,NaN,NaN,NaN,2008
94,Mar-2008,3.609,NaN,NaN,NaN,2008
95,Apr-2008,3.846,NaN,NaN,NaN,2008
96,May-2008,4.015,NaN,NaN,NaN,2008
...,...,...,...,...,...,...
291,Aug-2024,4.451,NaN,NaN,NaN,2024
292,Sep-2024,4.574,NaN,NaN,NaN,2024
293,Oct-2024,4.513,NaN,NaN,NaN,2024
294,Nov-2024,4.355,NaN,NaN,NaN,2024


In [22]:
gas_price_new['Date'] = pd.to_datetime(gas_price_new['Date'], format='%b-%Y')


gas_price_new['Fiscal_Year'] = gas_price_new['Date'].apply(lambda x: x.year + 1 if x.month > 12 else x.year)
gas_price_new['Fiscal_Quarter'] = gas_price_new['Date'].apply(
    lambda x: 'Q1' if x.month in [1, 2, 3]
    else 'Q2' if x.month in [4, 5, 6]
    else 'Q3' if x.month in [7, 8, 9]
    else 'Q4'
)

In [23]:
gas_price_new['County_Year_Quarter'] = gas_price_new['Fiscal_Year'].astype(str) + gas_price_new['Fiscal_Quarter']
gp_quarterly_avg = gas_price_new.groupby('County_Year_Quarter')['California All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon)'].mean().reset_index()
gp_quarterly_avg.rename(columns={'California All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon)': 'Quarterly Average Price'}, inplace=True)
gp_quarterly_avg

,County_Year_Quarter,Quarterly Average Price
0,2008Q1,3.378667
1,2008Q2,4.130667
2,2008Q3,4.160333
3,2008Q4,2.606000
4,2009Q1,2.185000
...,...,...
63,2023Q4,5.013000
64,2024Q1,4.616000
65,2024Q2,5.050000
66,2024Q3,4.538333


In [24]:
counties_df = pd.DataFrame({'County': counties})  # 52 counties
gp_quarterly_avg['key'] = 1
counties_df['key'] = 1
merged_df = pd.merge(gp_quarterly_avg, counties_df, on='key').drop('key', axis=1)
merged_df["County_Quarter"] = merged_df['County'] + "_" + merged_df['County_Year_Quarter']
gas_prices = merged_df[["County_Quarter", "Quarterly Average Price"]]
gas_prices["County_Quarter"] = gas_prices["County_Quarter"].str.upper()

/tmp/ipykernel_1160/1156894570.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gas_prices["County_Quarter"] = gas_prices["County_Quarter"].str.upper()


In [25]:
gas_prices

,County_Quarter,Quarterly Average Price
0,ALAMEDA_2008Q1,3.378667
1,BUTTE_2008Q1,3.378667
2,CALAVERAS_2008Q1,3.378667
3,COLUSA_2008Q1,3.378667
4,CONTRA COSTA_2008Q1,3.378667
...,...,...
3531,VENTURA_2024Q4,4.370333
3532,YOLO_2024Q4,4.370333
3533,MADERA_2024Q4,4.370333
3534,TEHAMA_2024Q4,4.370333


In [26]:
## List of all y tables 
## 1. elec_expanded
## 2. ng_expanded
## 3. ZEVLDV_new
## 4. gas_prices

In [27]:
Elecng = pd.merge(ng_expanded, elec_expanded, on="County_Quarter") 
ZEVgas =  pd.merge(ZEVLDV_new, gas_prices, on="County_Quarter")
ytables = pd.merge(ZEVgas, Elecng, on="County_Quarter")

In [28]:
ytables

,County_Quarter,ZEV Sales,LDV Sales,ZEV Sales Share,Quarterly Average Price,MMTherms,GWh
0,ALAMEDA_2023Q1,6954,16351,43%,4.581667,3.966175e+08,11314.010710
1,BUTTE_2023Q1,139,1254,11%,4.581667,4.187573e+07,1340.902019
2,CALAVERAS_2023Q1,42,330,13%,4.581667,9.451130e+05,298.698505
3,COLUSA_2023Q1,4,129,3%,4.581667,3.304655e+07,265.463978
4,CONTRA COSTA_2023Q1,3531,11275,31%,4.581667,9.759170e+08,8033.684516
...,...,...,...,...,...,...,...
2771,TEHAMA_2008Q4,0,100,0%,2.606000,1.175910e+07,508.706976
2772,TRINITY_2008Q4,0,25,0%,2.606000,6.122900e+04,12.383056
2773,TULARE_2008Q4,0,874,0%,2.606000,1.464699e+08,3870.941268
2774,VENTURA_2008Q4,1,3649,0%,2.606000,2.046721e+08,5907.359909


In [29]:
ytables = ytables[['County_Quarter', 'ZEV Sales', 'LDV Sales', 'Quarterly Average Price', ' MMTherms ', 'GWh']]
ytables

,County_Quarter,ZEV Sales,LDV Sales,Quarterly Average Price,MMTherms,GWh
0,ALAMEDA_2023Q1,6954,16351,4.581667,3.966175e+08,11314.010710
1,BUTTE_2023Q1,139,1254,4.581667,4.187573e+07,1340.902019
2,CALAVERAS_2023Q1,42,330,4.581667,9.451130e+05,298.698505
3,COLUSA_2023Q1,4,129,4.581667,3.304655e+07,265.463978
4,CONTRA COSTA_2023Q1,3531,11275,4.581667,9.759170e+08,8033.684516
...,...,...,...,...,...,...
2771,TEHAMA_2008Q4,0,100,2.606000,1.175910e+07,508.706976
2772,TRINITY_2008Q4,0,25,2.606000,6.122900e+04,12.383056
2773,TULARE_2008Q4,0,874,2.606000,1.464699e+08,3870.941268
2774,VENTURA_2008Q4,1,3649,2.606000,2.046721e+08,5907.359909


In [30]:
ytables = ytables.rename(columns={'Quarterly Average Price': 'Gas Price', " MMTherms ": "NG Consumption(MMTherms)", "GWh": "Electricity Consumption(GWh)"})
ytables

,County_Quarter,ZEV Sales,LDV Sales,Gas Price,NG Consumption(MMTherms),Electricity Consumption(GWh)
0,ALAMEDA_2023Q1,6954,16351,4.581667,3.966175e+08,11314.010710
1,BUTTE_2023Q1,139,1254,4.581667,4.187573e+07,1340.902019
2,CALAVERAS_2023Q1,42,330,4.581667,9.451130e+05,298.698505
3,COLUSA_2023Q1,4,129,4.581667,3.304655e+07,265.463978
4,CONTRA COSTA_2023Q1,3531,11275,4.581667,9.759170e+08,8033.684516
...,...,...,...,...,...,...
2771,TEHAMA_2008Q4,0,100,2.606000,1.175910e+07,508.706976
2772,TRINITY_2008Q4,0,25,2.606000,6.122900e+04,12.383056
2773,TULARE_2008Q4,0,874,2.606000,1.464699e+08,3870.941268
2774,VENTURA_2008Q4,1,3649,2.606000,2.046721e+08,5907.359909


In [31]:
total_aqi_avg

,County_Quarter,Daily AQI Value
0,ALAMEDA_2008Q1,40.465021
1,ALAMEDA_2008Q2,47.481481
2,ALAMEDA_2008Q3,49.563063
3,ALAMEDA_2008Q4,52.891051
4,ALAMEDA_2009Q1,40.027211
...,...,...
3447,YOLO_2023Q4,39.654639
3448,YOLO_2024Q1,22.896679
3449,YOLO_2024Q2,27.703704
3450,YOLO_2024Q3,38.584906


In [32]:
all_data = pd.merge(ytables, total_aqi_avg, on='County_Quarter', how='inner')
all_data = all_data[ytables.columns.tolist() + ['Daily AQI Value']]


In [33]:
all_data
all_data['ZEV_to_LDV_Ratio'] = all_data['ZEV Sales'] / all_data['LDV Sales']
all_data['Total_Energy_Consumption'] = all_data['NG Consumption(MMTherms)'] + all_data['Electricity Consumption(GWh)']
all_data['ZEV_GasPrice_Ratio'] = all_data['ZEV Sales'] / all_data['Gas Price']
all_data

,County_Quarter,ZEV Sales,LDV Sales,Gas Price,NG Consumption(MMTherms),Electricity Consumption(GWh),Daily AQI Value,ZEV_to_LDV_Ratio,Total_Energy_Consumption,ZEV_GasPrice_Ratio
0,ALAMEDA_2023Q1,6954,16351,4.581667,3.966175e+08,11314.010710,31.319527,0.425295,3.966288e+08,1517.788287
1,BUTTE_2023Q1,139,1254,4.581667,4.187573e+07,1340.902019,23.716418,0.110845,4.187707e+07,30.338305
2,CALAVERAS_2023Q1,42,330,4.581667,9.451130e+05,298.698505,23.734940,0.127273,9.454117e+05,9.166970
3,COLUSA_2023Q1,4,129,4.581667,3.304655e+07,265.463978,26.400000,0.031008,3.304681e+07,0.873045
4,CONTRA COSTA_2023Q1,3531,11275,4.581667,9.759170e+08,8033.684516,33.821429,0.313171,9.759250e+08,770.680247
...,...,...,...,...,...,...,...,...,...,...
2739,SUTTER_2008Q4,0,227,2.606000,2.115272e+07,626.845195,55.088235,0.000000,2.115335e+07,0.000000
2740,TRINITY_2008Q4,0,25,2.606000,6.122900e+04,12.383056,17.586207,0.000000,6.124138e+04,0.000000
2741,TULARE_2008Q4,0,874,2.606000,1.464699e+08,3870.941268,73.084337,0.000000,1.464738e+08,0.000000
2742,VENTURA_2008Q4,1,3649,2.606000,2.046721e+08,5907.359909,45.445312,0.000274,2.046780e+08,0.383730


In [34]:
all_data.to_csv('all_data.csv')


In [35]:
cols = all_data.columns.difference(["County_Quarter", "Daily AQI Value"])
x = all_data[cols].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
df

,0,1,2,3,4,5,6,7
0,0.157127,0.636171,0.095268,0.129384,0.129385,0.207533,0.218511,0.980630
1,0.018471,0.636171,0.007209,0.013661,0.013661,0.004148,0.004368,0.255583
2,0.003981,0.636171,0.001820,0.000308,0.000308,0.001253,0.001320,0.293461
3,0.003519,0.636171,0.000647,0.010780,0.010780,0.000119,0.000126,0.071497
4,0.111521,0.636171,0.065661,0.318362,0.318357,0.105378,0.110952,0.722098
...,...,...,...,...,...,...,...,...
2739,0.008543,0.111750,0.001219,0.006900,0.006900,0.000000,0.000000,0.000000
2740,0.000000,0.111750,0.000041,0.000020,0.000020,0.000000,0.000000,0.000000
2741,0.053646,0.111750,0.004993,0.047781,0.047781,0.000000,0.000000,0.000000
2742,0.081958,0.111750,0.021179,0.066768,0.066768,0.000030,0.000055,0.000632


In [36]:
df["Daily AQI Value"] = all_data["Daily AQI Value"]
df

,0,1,2,3,4,5,6,7,Daily AQI Value
0,0.157127,0.636171,0.095268,0.129384,0.129385,0.207533,0.218511,0.980630,31.319527
1,0.018471,0.636171,0.007209,0.013661,0.013661,0.004148,0.004368,0.255583,23.716418
2,0.003981,0.636171,0.001820,0.000308,0.000308,0.001253,0.001320,0.293461,23.734940
3,0.003519,0.636171,0.000647,0.010780,0.010780,0.000119,0.000126,0.071497,26.400000
4,0.111521,0.636171,0.065661,0.318362,0.318357,0.105378,0.110952,0.722098,33.821429
...,...,...,...,...,...,...,...,...,...
2739,0.008543,0.111750,0.001219,0.006900,0.006900,0.000000,0.000000,0.000000,55.088235
2740,0.000000,0.111750,0.000041,0.000020,0.000020,0.000000,0.000000,0.000000,17.586207
2741,0.053646,0.111750,0.004993,0.047781,0.047781,0.000000,0.000000,0.000000,73.084337
2742,0.081958,0.111750,0.021179,0.066768,0.066768,0.000030,0.000055,0.000632,45.445312


In [37]:
df.columns = ['ZEV Sales', "LDV Sales", "Gas Price", "NG Consumption(MMTherms)", "Electricity Consumption(GWh)", "ZEV_to_LDV_Ratio", "Total_Energy_Consumption", "ZEV_GasPrice_Ratio","Daily AQI Value"]

In [38]:
df

,ZEV Sales,LDV Sales,Gas Price,NG Consumption(MMTherms),Electricity Consumption(GWh),ZEV_to_LDV_Ratio,Total_Energy_Consumption,ZEV_GasPrice_Ratio,Daily AQI Value
0,0.157127,0.636171,0.095268,0.129384,0.129385,0.207533,0.218511,0.980630,31.319527
1,0.018471,0.636171,0.007209,0.013661,0.013661,0.004148,0.004368,0.255583,23.716418
2,0.003981,0.636171,0.001820,0.000308,0.000308,0.001253,0.001320,0.293461,23.734940
3,0.003519,0.636171,0.000647,0.010780,0.010780,0.000119,0.000126,0.071497,26.400000
4,0.111521,0.636171,0.065661,0.318362,0.318357,0.105378,0.110952,0.722098,33.821429
...,...,...,...,...,...,...,...,...,...
2739,0.008543,0.111750,0.001219,0.006900,0.006900,0.000000,0.000000,0.000000,55.088235
2740,0.000000,0.111750,0.000041,0.000020,0.000020,0.000000,0.000000,0.000000,17.586207
2741,0.053646,0.111750,0.004993,0.047781,0.047781,0.000000,0.000000,0.000000,73.084337
2742,0.081958,0.111750,0.021179,0.066768,0.066768,0.000030,0.000055,0.000632,45.445312


In [39]:
train_df, test_df = train_test_split(
    df,
    train_size=0.75,
    random_state= 42,    
)

In [40]:
train_df

,ZEV Sales,LDV Sales,Gas Price,NG Consumption(MMTherms),Electricity Consumption(GWh),ZEV_to_LDV_Ratio,Total_Energy_Consumption,ZEV_GasPrice_Ratio,Daily AQI Value
1106,0.016439,0.238276,0.012908,0.017181,0.017181,0.004685,0.007332,0.162261,26.776536
2501,0.003494,0.229251,0.000356,0.008878,0.008878,0.000000,0.000000,0.000000,50.451923
788,0.214043,0.385330,0.152600,0.147691,0.147693,0.039782,0.052770,0.117402,47.157809
969,0.077145,0.371085,0.036817,0.080085,0.080085,0.009699,0.013059,0.118384,56.454545
2743,0.022275,0.111750,0.005500,0.020163,0.020163,0.000000,0.000000,0.000000,51.163462
...,...,...,...,...,...,...,...,...,...
1638,0.016267,0.472217,0.010190,0.008958,0.008958,0.001880,0.002288,0.082302,28.233333
1095,0.218422,0.238276,0.157179,0.128283,0.128286,0.017787,0.027834,0.050964,52.924307
1130,0.018744,0.242700,0.020473,0.022930,0.022930,0.009938,0.015468,0.217636,48.231405
1294,0.132631,0.168466,0.078155,0.370634,0.370629,0.021458,0.036711,0.123563,31.736842


In [41]:
x_columns_to_drop = ['Daily AQI Value']
x_train = train_df.drop(x_columns_to_drop, axis=1)
x_train

,ZEV Sales,LDV Sales,Gas Price,NG Consumption(MMTherms),Electricity Consumption(GWh),ZEV_to_LDV_Ratio,Total_Energy_Consumption,ZEV_GasPrice_Ratio
1106,0.016439,0.238276,0.012908,0.017181,0.017181,0.004685,0.007332,0.162261
2501,0.003494,0.229251,0.000356,0.008878,0.008878,0.000000,0.000000,0.000000
788,0.214043,0.385330,0.152600,0.147691,0.147693,0.039782,0.052770,0.117402
969,0.077145,0.371085,0.036817,0.080085,0.080085,0.009699,0.013059,0.118384
2743,0.022275,0.111750,0.005500,0.020163,0.020163,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
1638,0.016267,0.472217,0.010190,0.008958,0.008958,0.001880,0.002288,0.082302
1095,0.218422,0.238276,0.157179,0.128283,0.128286,0.017787,0.027834,0.050964
1130,0.018744,0.242700,0.020473,0.022930,0.022930,0.009938,0.015468,0.217636
1294,0.132631,0.168466,0.078155,0.370634,0.370629,0.021458,0.036711,0.123563


In [42]:
collums_to_drop = ['ZEV Sales', 'LDV Sales', "Gas Price" , "NG Consumption(MMTherms)", "Electricity Consumption(GWh)", "ZEV_to_LDV_Ratio", 'Total_Energy_Consumption', 'ZEV_GasPrice_Ratio']
y_train = train_df.drop(collums_to_drop, axis=1)
y_train

,Daily AQI Value
1106,26.776536
2501,50.451923
788,47.157809
969,56.454545
2743,51.163462
...,...
1638,28.233333
1095,52.924307
1130,48.231405
1294,31.736842


In [43]:
trainx_columns_to_drop = ['Daily AQI Value']
x_test = test_df.drop(x_columns_to_drop, axis=1)
x_test

,ZEV Sales,LDV Sales,Gas Price,NG Consumption(MMTherms),Electricity Consumption(GWh),ZEV_to_LDV_Ratio,Total_Energy_Consumption,ZEV_GasPrice_Ratio
859,0.005370,0.319236,0.000962,0.002618,0.002618,0.000090,0.000127,0.037799
1521,0.036682,0.175633,0.017446,0.033645,0.033645,0.002059,0.003490,0.052874
1322,0.016543,0.168466,0.013853,0.016855,0.016855,0.003939,0.006740,0.127188
1352,0.280458,0.162184,0.257615,0.185924,0.185926,0.072490,0.125070,0.126759
2031,0.017182,0.479915,0.006702,0.009627,0.009627,0.000925,0.001118,0.061250
...,...,...,...,...,...,...,...,...
764,0.056182,0.455229,0.018181,0.050607,0.050607,0.001343,0.001661,0.033097
1778,0.042212,0.493452,0.005979,0.040392,0.040392,0.000149,0.000178,0.011054
657,0.266231,0.251283,0.243476,0.193969,0.193971,0.150233,0.231420,0.277951
1614,0.202614,0.534242,0.119936,0.152837,0.152838,0.010565,0.012141,0.039662


In [44]:
collums_to_drop = ['ZEV Sales', 'LDV Sales', "Gas Price" , "NG Consumption(MMTherms)", "Electricity Consumption(GWh)", "ZEV_to_LDV_Ratio", 'Total_Energy_Consumption',  'ZEV_GasPrice_Ratio']
y_test = test_df.drop(collums_to_drop, axis=1)
y_test

,Daily AQI Value
859,30.186667
1521,25.599291
1322,27.467391
1352,49.395455
2031,7.925926
...,...
764,45.534694
1778,41.320000
657,61.634409
1614,48.903704


In [45]:
reg = LinearRegression().fit(x_train, y_train)
reg.score(x_train,y_train)
y_pred = reg.predict(x_test)
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
RMSE


12.870550304441105

In [46]:
def VIF(df, columns):
    values = sm.add_constant(df[columns]).values
    num_columns = len(columns)+1
    vif = [variance_inflation_factor(values, i) for i in range(num_columns)]
    return pd.Series(vif[1:], index=columns)

In [47]:
columns = ["ZEV Sales", "LDV Sales", "Gas Price", "ZEV_to_LDV_Ratio", 'ZEV_GasPrice_Ratio']
VIF(all_data, columns)

ZEV Sales             36.401456
LDV Sales              2.886250
Gas Price              1.585088
ZEV_to_LDV_Ratio       1.978741
ZEV_GasPrice_Ratio    44.841377
dtype: float64

In [48]:
regr = make_pipeline(StandardScaler(), SVR(C=300.0, epsilon=0.99))
regr.fit(x_train, y_train)
y_pred = regr.predict(x_test)
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
RMSE

/root/venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


12.370079162106668

In [49]:
xgb_r = xg.XGBRegressor(objective ='reg:gamma', n_estimators = 110, seed = 42)

xgb_r.fit(x_train, y_train)
y_pred = xgb_r.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

8.996160174901798

In [50]:
rfr = RandomForestRegressor(max_depth=15, random_state=42)
rfr.fit(x_train, y_train)
y_pred = rfr.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

/tmp/ipykernel_1160/3298888921.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfr.fit(x_train, y_train)


10.592537314297676

In [52]:
ridge = Ridge(alpha=.0000001)
ridge.fit(x_train, np.ravel(y_train))
y_pred = ridge.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

12.870550311371462

In [53]:
lasso = Lasso(alpha=0.0000001, max_iter=8000)
lasso.fit(x_train, np.ravel(y_train))
y_pred = lasso.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

12.870550629639876

In [54]:
elastic = ElasticNet(alpha=0.00001, l1_ratio=0.05, max_iter=5000)
elastic.fit(x_train, np.ravel(y_train))
y_pred = elastic.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

/root/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.131e+04, tolerance: 3.824e+01
  model = cd_fast.enet_coordinate_descent(


12.871982620643553

In [55]:
xgb_r = xg.XGBRegressor(
    n_estimators=500,       # more boosting rounds
    max_depth=6,            # moderate complexity
    learning_rate=0.05,     # slower learning, better accuracy
    subsample=0.8,          # random row sampling
    colsample_bytree=0.8,   # random feature sampling
    reg_lambda=1,           # regularization to prevent overfitting
    random_state=42
)

xgb_r.fit(x_train, y_train)
y_pred = xgb_r.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

8.994194852895381

In [56]:
xgb_r = xg.XGBRegressor(
    n_estimators=800,
    learning_rate=0.03,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb_r.fit(x_train, y_train)
y_pred = xgb_r.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

8.823297172340594

In [57]:

all_data['County'] = all_data['County_Quarter'].str.split('_').str[0]

all_data['Year'] = all_data['County_Quarter'].str.extract(r'_(\d{4})Q\d').astype(int)
all_data['Quarter'] = all_data['County_Quarter'].str.extract(r'(\d)').astype(int)


all_data = all_data.drop('County_Quarter', axis=1)
all_data

,ZEV Sales,LDV Sales,Gas Price,NG Consumption(MMTherms),Electricity Consumption(GWh),Daily AQI Value,ZEV_to_LDV_Ratio,Total_Energy_Consumption,ZEV_GasPrice_Ratio,County,Year,Quarter
0,6954,16351,4.581667,3.966175e+08,11314.010710,31.319527,0.425295,3.966288e+08,1517.788287,ALAMEDA,2023,2
1,139,1254,4.581667,4.187573e+07,1340.902019,23.716418,0.110845,4.187707e+07,30.338305,BUTTE,2023,2
2,42,330,4.581667,9.451130e+05,298.698505,23.734940,0.127273,9.454117e+05,9.166970,CALAVERAS,2023,2
3,4,129,4.581667,3.304655e+07,265.463978,26.400000,0.031008,3.304681e+07,0.873045,COLUSA,2023,2
4,3531,11275,4.581667,9.759170e+08,8033.684516,33.821429,0.313171,9.759250e+08,770.680247,CONTRA COSTA,2023,2
...,...,...,...,...,...,...,...,...,...,...,...,...
2739,0,227,2.606000,2.115272e+07,626.845195,55.088235,0.000000,2.115335e+07,0.000000,SUTTER,2008,2
2740,0,25,2.606000,6.122900e+04,12.383056,17.586207,0.000000,6.124138e+04,0.000000,TRINITY,2008,2
2741,0,874,2.606000,1.464699e+08,3870.941268,73.084337,0.000000,1.464738e+08,0.000000,TULARE,2008,2
2742,1,3649,2.606000,2.046721e+08,5907.359909,45.445312,0.000274,2.046780e+08,0.383730,VENTURA,2008,2


In [58]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
all_data["County_ID"] = le.fit_transform(all_data["County"])

all_data.drop('County', axis = 1)

,ZEV Sales,LDV Sales,Gas Price,NG Consumption(MMTherms),Electricity Consumption(GWh),Daily AQI Value,ZEV_to_LDV_Ratio,Total_Energy_Consumption,ZEV_GasPrice_Ratio,Year,Quarter,County_ID
0,6954,16351,4.581667,3.966175e+08,11314.010710,31.319527,0.425295,3.966288e+08,1517.788287,2023,2,0
1,139,1254,4.581667,4.187573e+07,1340.902019,23.716418,0.110845,4.187707e+07,30.338305,2023,2,1
2,42,330,4.581667,9.451130e+05,298.698505,23.734940,0.127273,9.454117e+05,9.166970,2023,2,2
3,4,129,4.581667,3.304655e+07,265.463978,26.400000,0.031008,3.304681e+07,0.873045,2023,2,3
4,3531,11275,4.581667,9.759170e+08,8033.684516,33.821429,0.313171,9.759250e+08,770.680247,2023,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...
2739,0,227,2.606000,2.115272e+07,626.845195,55.088235,0.000000,2.115335e+07,0.000000,2008,2,39
2740,0,25,2.606000,6.122900e+04,12.383056,17.586207,0.000000,6.124138e+04,0.000000,2008,2,41
2741,0,874,2.606000,1.464699e+08,3870.941268,73.084337,0.000000,1.464738e+08,0.000000,2008,2,42
2742,1,3649,2.606000,2.046721e+08,5907.359909,45.445312,0.000274,2.046780e+08,0.383730,2008,2,43


In [59]:
train_df, test_df = train_test_split(
    all_data,
    train_size=0.75,
    random_state= 42,    
)

In [60]:
x_columns_to_drop = ['Daily AQI Value', 'County']
x_train = train_df.drop(x_columns_to_drop, axis=1)
x_train

,ZEV Sales,LDV Sales,Gas Price,NG Consumption(MMTherms),Electricity Consumption(GWh),ZEV_to_LDV_Ratio,Total_Energy_Consumption,ZEV_GasPrice_Ratio,Year,Quarter,County_ID
1106,157,2231,3.082667,5.266696e+07,1194.810217,0.070372,5.266816e+07,50.929931,2017,2,34
2501,0,79,3.048667,2.721529e+07,263.690469,0.000000,2.721555e+07,0.000000,2009,2,3
788,1333,26180,3.636667,4.527382e+08,15407.780678,0.050917,4.527536e+08,366.544455,2019,2,23
969,325,6330,3.583000,2.454952e+08,5561.142347,0.051343,2.455007e+08,90.706112,2018,2,29
2743,0,961,2.606000,6.180875e+07,1614.529342,0.000000,6.181036e+07,0.000000,2008,2,44
...,...,...,...,...,...,...,...,...,...,...,...
1638,63,1765,3.964000,2.746004e+07,1182.414110,0.035694,2.746123e+07,15.893037,2014,2,5
1095,596,26965,3.082667,3.932446e+08,15722.740840,0.022103,3.932603e+08,193.339100,2017,2,23
1130,333,3528,3.099333,7.029147e+07,1360.574290,0.094388,7.029283e+07,107.442461,2017,2,14
1294,719,13417,2.819667,1.136154e+09,9552.083382,0.053589,1.136164e+09,254.994680,2016,2,4


In [61]:
collums_to_drop = ['ZEV Sales', 'LDV Sales', "Gas Price" , "NG Consumption(MMTherms)", "Electricity Consumption(GWh)", "ZEV_to_LDV_Ratio", 'Total_Energy_Consumption', 'ZEV_GasPrice_Ratio', 'County', 'Year', 'Quarter', 'County_ID']
y_train = train_df.drop(collums_to_drop, axis=1)
y_train

,Daily AQI Value
1106,26.776536
2501,50.451923
788,47.157809
969,56.454545
2743,51.163462
...,...
1638,28.233333
1095,52.924307
1130,48.231405
1294,31.736842


In [62]:
trainx_columns_to_drop = ['Daily AQI Value', 'County']
x_test = test_df.drop(x_columns_to_drop, axis=1)
x_test

,ZEV Sales,LDV Sales,Gas Price,NG Consumption(MMTherms),Electricity Consumption(GWh),ZEV_to_LDV_Ratio,Total_Energy_Consumption,ZEV_GasPrice_Ratio,Year,Quarter,County_ID
859,3,183,3.387667,8.025895e+06,398.644719,0.016393,8.026294e+06,0.885565,2018,2,6
1521,69,3009,2.846667,1.031355e+08,2650.785723,0.022931,1.031382e+08,24.238876,2015,2,18
1322,132,2393,2.819667,5.166709e+07,1202.259783,0.055161,5.166829e+07,46.814044,2016,2,34
1352,2429,44184,2.796000,5.699368e+08,20184.726856,0.054975,5.699570e+08,868.741059,2016,2,21
2031,31,1167,3.993000,2.951179e+07,1248.236244,0.026564,2.951304e+07,7.763586,2012,2,5
...,...,...,...,...,...,...,...,...,...,...,...
764,45,3135,3.900000,1.551313e+08,4053.346307,0.014354,1.551353e+08,11.538462,2019,2,42
1778,5,1043,4.044000,1.238190e+08,3048.530157,0.004794,1.238221e+08,1.236400,2013,2,17
657,5034,41760,3.131667,5.945990e+08,19161.444914,0.120546,5.946182e+08,1607.450772,2020,2,21
1614,354,20580,4.197667,4.685114e+08,14585.723002,0.017201,4.685260e+08,84.332566,2014,2,26


In [63]:
collums_to_drop = ['ZEV Sales', 'LDV Sales', "Gas Price" , "NG Consumption(MMTherms)", "Electricity Consumption(GWh)", "ZEV_to_LDV_Ratio", 'Total_Energy_Consumption',  'ZEV_GasPrice_Ratio', 'County', 'Year', 'Quarter', 'County_ID']
y_test = test_df.drop(collums_to_drop, axis=1)
y_test

,Daily AQI Value
859,30.186667
1521,25.599291
1322,27.467391
1352,49.395455
2031,7.925926
...,...
764,45.534694
1778,41.320000
657,61.634409
1614,48.903704


In [64]:
xgb_r = xg.XGBRegressor(
    n_estimators=800,
    learning_rate=0.03,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb_r.fit(x_train, y_train)
y_pred = xgb_r.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

8.610992554953832

In [68]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, GroupKFold, train_test_split
from sklearn.metrics import mean_squared_error, make_scorer

# -----------------------------
# 1) Prepare X, y
#    (Adjust feature_cols to match your dataframe columns)
# -----------------------------
feature_cols = [
    "County_ID", "Year", "Quarter",
    "ZEV Sales", "LDV Sales", "Gas Price",
    "NG Consumption(MMTherms)", "Electricity Consumption(GWh)",
    "ZEV_to_LDV_Ratio", "Total_Energy_Consumption"
]
target_col = "Daily AQI Value"

X = all_data[feature_cols].copy()
y = all_data[target_col].copy()

# Optional: final hold-out test split (so GridSearchCV tunes only on train part)
X_train, X_hold, y_train, y_hold = train_test_split(
    X, y, test_size=0.15, random_state=42
)

# -----------------------------
# 2) CV strategy
#    Group by Year to reduce time leakage across quarters
# -----------------------------
groups = all_data.loc[X_train.index, "Year"]   # groups must align with X_train indexing
cv = GroupKFold(n_splits=5)

# -----------------------------
# 3) Model + scoring
# -----------------------------
model = XGBRegressor(
    objective="reg:squarederror",
    tree_method="hist",        # faster
    random_state=42
)

rmse_scorer = make_scorer(
    lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)),
    greater_is_better=False
)

# -----------------------------
# 4) SMALL grid (fast to run)
#    Expand if needed (see "Bigger grid" below)
# -----------------------------
param_grid = {
    "n_estimators": [300, 600, 900],
    "max_depth": [4, 6, 8],
    "learning_rate": [0.02, 0.04, 0.08],
    "subsample": [0.7, 0.85, 1.0],
    "colsample_bytree": [0.7, 0.85, 1.0],
    "reg_lambda": [0.5, 1, 5],
    "min_child_weight": [1, 3, 5],
}

grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=rmse_scorer,    # negative RMSE (lower is better)
    cv=cv,
    n_jobs=-1,
    verbose=2
)
grid.fit(X_train, y_train, groups=groups)

print("Best params:", grid.best_params_)
print("Best CV RMSE:", -grid.best_score_)

# -----------------------------
# 5) Evaluate on the final hold-out set
# -----------------------------
best_model = grid.best_estimator_
y_pred = best_model.predict(X_hold)
rmse_hold = np.sqrt(mean_squared_error(y_hold, y_pred))
print("Hold-out RMSE:", rmse_hold)

# Optional: quick feature importance peek
imp = pd.Series(best_model.feature_importances_, index=feature_cols).sort_values(ascending=False)
print(imp)

[CV] END colsample_bytree=1.0, learning_rate=0.08, max_depth=4, min_child_weight=3, n_estimators=900, reg_lambda=0.5, subsample=0.85; total time=   0.4s
[CV] END colsample_bytree=1.0, learning_rate=0.08, max_depth=4, min_child_weight=3, n_estimators=900, reg_lambda=0.5, subsample=0.85; total time=   0.4s
[CV] END colsample_bytree=1.0, learning_rate=0.08, max_depth=4, min_child_weight=3, n_estimators=900, reg_lambda=0.5, subsample=0.85; total time=   0.4s
[CV] END colsample_bytree=1.0, learning_rate=0.08, max_depth=4, min_child_weight=3, n_estimators=900, reg_lambda=0.5, subsample=0.85; total time=   0.5s
[CV] END colsample_bytree=1.0, learning_rate=0.08, max_depth=4, min_child_weight=3, n_estimators=900, reg_lambda=0.5, subsample=0.85; total time=   0.4s
[CV] END colsample_bytree=1.0, learning_rate=0.08, max_depth=4, min_child_weight=3, n_estimators=900, reg_lambda=0.5, subsample=1.0; total time=   0.5s
[CV] END colsample_bytree=1.0, learning_rate=0.08, max_depth=4, min_child_weight=3,

In [ ]:
##Best params: {'colsample_bytree': 0.7, 'learning_rate': 0.02, 'max_depth': 8, 'min_child_weight': 1, 'n_estimators': 300, 'reg_lambda': 5, 'subsample': 0.7}
##Best CV RMSE: 11.847167584187211
##Hold-out RMSE: 8.683033003419897
##Electricity Consumption(GWh)    0.225236
##NG Consumption(MMTherms)        0.125444
##Total_Energy_Consumption        0.121068
##Gas Price                       0.118813
##County_ID                       0.117281
##ZEV_to_LDV_Ratio                0.085801
##Year                            0.084343
##LDV Sales                       0.072649
##ZEV Sales                       0.049366
##Quarter                         0.000000
##dtype: float32

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3e090f6e-56da-4d2e-a486-5a0a57b70d9d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>